<center><font size=6><b>GEOS 669 Geodetic Methods and Modeling</b></font></center>

# Lab 9: Modeling - Plate Motion

Today you will:

 * plot contiguous US velocities in ITRF reference frame
 * determine the Euler Pole for the North American Plate for a subset of these sites
 * forward model the plate velocities at all contiguous US sites and remove them from the ITRF velocities to determine site motion with respect to stable North America
 * Bonus: compare to official NOAM velocities

In [ ]:
#SETUP - install necessary packages
!conda install -y gmt pygmt -c conda-forge

In [ ]:
#Download the most recent site velocities provided by UNAVCO
!wget https://data.unavco.org/archive/gnss/products/velocity/cwu.final_igs14.vel

Now import some needed packages. We also define a function that plots velocities on a map using pygmt (example of its use shown further below.)

In [ ]:
import numpy as np
import pandas as pd
import pygmt
import matplotlib.pyplot as plt

#simple routine to plot the horizontal (and optional vertical) velocity fields
#using pygmt. returns the figure handle, which means that more pygmt calls
#can be issues on that, or the result can be plotted / saved.
def plot_velocities(elong, nlat, evel, nvel, uvel=None, reg="-135/-110/20/55"):
    
    pygmt.config(MAP_FRAME_TYPE="plain")

    vec_spec = "e90/0.95/10"
    
    fig = pygmt.Figure()
    
    fig.coast( 
        region=reg,
        projection="M21c",
        land="lightgray",
        water="white",
        shorelines="1/0.5p",
        frame=True
    )

    df = pd.DataFrame(
        data={
            "x": elong.flatten(),
            "y": nlat.flatten(),
            "east_velocity":  evel.flatten(),
            "north_velocity": nvel.flatten(),
        }
    )
    
    fig.velo(
        data=df,
        pen="0.2p,red",
        line=True,
        spec=vec_spec,
        vector="0.3c+p1p+e+gred",
        frame=True
    )
    
    #plot optional vertical velocities
    if uvel is not None:
        df = pd.DataFrame(
            data={
                "x": elong,
                "y": nlat,
                "east_velocity":  uvel.flatten()*0,
                "north_velocity": uvel.flatten(),
            }
        )
    
        fig.velo(
            data=df,
            pen="0.2p,blue",
            line=True,
            spec=vec_spec,
            vector="0.3c+p1p+e+gblue",
            frame=True
        )

    
    #station locations
    fig.plot(x=elon, y=nlat, style='c0.04i', color='black', pen='0.4p,white');
    
    return fig


# 1) Load, Subset and Plot Data

Now, we load the data into a pandas dataframe. We're forming a subset for just Alaska stations and use the most recent velocity observations (there are some duplicate stations in the record that give velocity observations for different time frames). Make sure to open the data file directly using the JupyterHub file browser; the header information will come in handy further down.

In [ ]:
#load and prepare velocity data; the first 35 lines are header information and use line 35 to name the columns.
cwu = pd.read_csv('cwu.final_igs14.vel', header=35, dtype=None, sep='\s+')

#create a roughly contiguous US subset: pick recent data to avoid station duplication, and pick a reasonable lat lon window
noam  = cwu[(cwu['last_epoch']>20200000000000) & (cwu['Ref_Nlat']>26) & (cwu['Ref_Nlat']<50) & (cwu['Ref_Elong']<300) & (cwu['Ref_Elong']>220)]

#make sure to reset the index to ensure the counting works well
noam = noam.reset_index()

#get the length of the data set
l    = noam.shape[0]

#output the dataframe to see what's in there 
#(if you want to see the full header information 
#open the file directly using the JupyterHub file browser)
noam

Now that the data are loaded into a pandas data frame, we can plot them. 

In [ ]:
#retrieve local velocities and lat lon for plotting. 
#make sure these are column vectors by reshaping.
evel = noam["dE/dt"].values
nvel = noam["dN/dt"].values
uvel = noam["dU/dt"].values

nlat = noam["Ref_Nlat"].values
elon = noam["Ref_Elong"].values

#this uses the function we defined above. It returns the figure handle,
#so we still have to display it (could save it, too, or otherwise manipulate)
f = plot_velocities(elon, nlat, evel, nvel, uvel=uvel, reg="-135/-66/25/50")
f.show()


# 2) Estimate Euler Pole

From the image above, select a latitude-longitude box that is likely to enclose stations with mostly plate motion driven rotation only. Note that you have the code to get that subset already given above for a larger lat-lon box, so you need to only copy this down here and change the values.

Now, determine the Euler Pole from these data! Note that you shouldn't use the X,Y,Z velocities given in the file, but generate **new** X,Y,Z velocities from the local east and north velocities. You do that by rotating the east and north velocities into XYZ:

$$
\begin{bmatrix}
               v_n \\
               v_e
        \end{bmatrix} = \begin{bmatrix}
                            -sin(\lambda)cos(\phi) & -sin(\lambda)sin(\phi) & cos(\lambda) \\
                            -sin(\phi)             &  cos(\phi)             & 0
             \end{bmatrix}  \begin{bmatrix}
                         v_x \\
                         v_y \\
                         v_z
             \end{bmatrix}
$$

where $\lambda, \phi$ are the station's latitude and longitude (in radians!), respectively. If you use the XYZ velocities given in the file directly, you bias the latitude of the Euler Pole.

Performing the rotation of $v_{en}$ into $v_{xyz}$ is probably most straight forward by doing this station by station and assembling three arrays `dXdt`, `dYdt`, and `dZdt` along the way that hold all the station velocities. Once you have that - extract the X, Y, Z coordinates (check for their units in the header information of the file!) for each station (three long arrays with ALL the coordinates will do).

Remember that together

$$
\begin{bmatrix}
               v_x \\
               v_y \\
               v_z
        \end{bmatrix} = \begin{bmatrix}
                             0 &  Z & -Y \\
                            -Z &  0 &  X \\ 
                             Y & -X &  0 
             \end{bmatrix}  \begin{bmatrix}
                         \omega_x \\
                         \omega_y \\
                         \omega_z
             \end{bmatrix}
$$

As you assemble $G$ and $d$, it's important to think vectorized, as we've done in prior labs. Once you have those components put together, it's again a simple call to `np.linalg.lstsq` to get the plate angular velocity vector $\omega$.

 * Print out your solution for the plate angular velocity. What's its unit?
 * Print the plate angular in units of deg/Myr.
 * Calculate the Euler Pole latitude (deg), longitude (deg) and angular speed (deg/Myr) from the plate angular velocity in the **original units (not in deg/Myr)** but give its values in the units specified in parentheses.

In [ ]:
#write your code here:

In the code below, add the Euler Pole to the plot of the data you've been given above:

 * uncomment all three lines
 * add reasonable values for W, E, S, N into the `reg` parameter to include the data and the Euler Pole in the map
 * either make sure your Euler Pole longitude and latitude variables are called `euler_phi` and `euler_lam`, respectively, or replace those strings with your values or variable names
 * run the cell.
 
**Does your Euler Pole location seem reasonable? How about the rotation speed (assuming right-hand-rule / counter clockwise is positive)? How does your North America (NOAM) Pole compare to published ones?**

In [ ]:
#f = plot_velocities(elon, nlat, evel, nvel, reg="W/E/S/N")
#f.plot(x=euler_phi, y=euler_lam, style='c0.15i', color='red', pen='0.4p,white');
f.show()

# 3) Remove Plate Velocity

Now for the fun part: Remove the plate velocity you predict for North America from all the horizontal data in the contiguous US. I am giving you below a data frame `noam` again using the original lat-lon box, from this I pull for you:

 * the lat lon station locations
 * the local east and north velocities
 * the X,Y,Z station locations
 
In the block that's commented out for your code:

 * set up the larger $G$ matrix (as before)
 * solve the forward problem with your calculated $\omega$ (in the same units as you've gotten out of the inversion before!)
 
Now you have a very long velocity vector that contains your plate motion predictions in ITRF reference frame, one third is X, one third is Y, one third is Z velocity; it might make sense to separate that vector into three individual vectors.

Almost there. We've got to do the rotation from ITRF into NEU now. You have all the code for that in a cell above and it will require only minimal adjustments, instead of solving for $v_{xyz}$ you are interested in $v_{ne}$, so you don't have to use the transpose / inverse of the rotation matrix. Throughout this loop over all stations and individual rotation, you will now assemble the east and north velocity vectors, which contain the plate velocity at each site.

Now, all you need to do is subtract the local plate velocities from the local ITRF velocities and you've got the motion of 1700+ sites in a North American reference frame. The plotting code for that is given. You've got to make sure you'll give it the correct variables.

**Interpret the velocities seen along the US West Coast! Are those what you would expect from what we talked about in class?**

In [ ]:
noam = cwu[(cwu['last_epoch']>20200000000000) & (cwu['Ref_Nlat']>26) & (cwu['Ref_Nlat']<50) & (cwu['Ref_Elong']<300) & (cwu['Ref_Elong']>220)]
l    = noam.shape[0]

#station longitudes and latitudes
elon = noam["Ref_Elong"].values
nlat = noam["Ref_Nlat"].values

#local station velocities
evel_all = noam["dE/dt"].values.reshape((l,1))
nvel_all = noam["dN/dt"].values.reshape((l,1))

#station XYZ coordinates
X    = noam["Ref_X"].values.reshape((l,1))
Y    = noam["Ref_Y"].values.reshape((l,1))
Z    = noam["Ref_Z"].values.reshape((l,1))

###
### YOUR CODE GOES HERE
###

#plot for all of US
f = plot_velocities(elon, nlat, evel, nvel, reg="-135/-66/25/50")
f.show()

#plot for west coast
f = plot_velocities(elon, nlat, evel, nvel, reg="-135/-110/25/50")
f.show()


# BONUS

Download the velocity field that's given in North American reference frame from UNAVCO. Retrieve the east and north velocities for the contiguous US stations and subtract your NOAM velocities from them. What do the residuals look like and why are they likley not zero?

In [ ]:
#get the file if you haven't yet

!wget https://data.unavco.org/archive/gnss/products/velocity/cwu.final_nam14.vel
    
## YOUR CODE GOES HERE